In [1]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
txt = pd.read_excel('guo(new_3w).xlsx').drop('空白字段',axis = 1)

In [3]:
for i in range(len(txt)):
    length = len(txt.loc[i]['商品描述'].split())
    if length >15:
        txt.drop(index = i,inplace=True)

In [4]:
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(txt['标签'])
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    if dic[i] > 200:
        frame = pd.concat([frame,data[:200]],axis = 0)
        continue
    elif dic[i]>=10:
        frame = pd.concat([frame,data],axis = 0)
frame = frame.reset_index(drop= True)
print('数据量：%d'%len(frame),'类目数：%d'%len(Counter(frame['标签'])))

数据量：21193 类目数：510


In [5]:
frematrix = CountVectorizer().fit_transform(frame['商品描述'])
tfidf = TfidfTransformer().fit_transform(frematrix)
X_train,X_test,Y_train,Y_test = train_test_split(tfidf,frame['标签'],test_size = 0.2)
lr = LogisticRegression()
lr.fit(X_train,Y_train)
y_pre = lr.predict(X_test)
np.mean(Y_test == y_pre)

0.8275536683179995

In [6]:
word_txt = []
for label in Counter(frame['标签']).keys():
    same = frame.loc[frame['标签'] == label]
    word = []
    for i in same['商品描述']:
        word += i.split()
    word_txt.append(word)

In [7]:
from gensim.models import word2vec

#sentences = word2vec.Text8Corpus('三万word2vec.txt')

#sentences = [i.split() for i in frame['商品描述']]

model = word2vec.Word2Vec(word_txt,size = 100,window = 1200 ,min_count = 3)

c:\python3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
word2vec_matrix = []
for line in range(frame.shape[0]):
    vector = []
    num = 0
    words = frame['商品描述'][line].split()
    for word in words:
        try:
            temp = model[word]
        except:
            continue 
        else:
            vector += list(temp)
            num += 1
    vector = vector + [0]*100*(15 - num)
    word2vec_matrix.append(vector)
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(frame['标签'])

c:\python3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [10]:
import tensorflow as tf
from tensorflow.contrib import rnn
import random

input_size = 100
timestep_size = 15 
hidden_size = 256
layer_num = 256
class_num = len(Counter(frame['标签']))
learning_rate = 0.001
with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32)
    batch_size = tf.placeholder(tf.int32, [], name='batch_size_input')
    _X = tf.placeholder(tf.float32,[None,1500])
    y = tf.placeholder(tf.int64,[None])
    x = tf.reshape(_X,[-1,timestep_size,input_size])

with tf.name_scope('weights'):
    W = tf.Variable(tf.truncated_normal([hidden_size, class_num], stddev=0.1), dtype=tf.float32)
with tf.name_scope('biases'):
    bias = tf.Variable(tf.constant(0.1,shape=[class_num]), dtype=tf.float32)
def attn_cell():
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size)
    with tf.name_scope('lstm_dropout'):
        return tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
enc_cells = []
for i in range(0, layer_num):
    enc_cells.append(attn_cell())
mlstm_cell = rnn.MultiRNNCell(enc_cells,state_is_tuple=True)
init_state = mlstm_cell.zero_state(batch_size,dtype=tf.float32)
outputs, state = tf.nn.dynamic_rnn(mlstm_cell, x, initial_state=init_state, dtype=tf.float32,time_major=False)

In [11]:
h_state = state[-1][1]
logits = tf.matmul(h_state, W) + bias #维度 batch_size * class_num
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits+1e-10,labels = y)
original_cost_function = tf.reduce_mean(loss)

tv = tf.trainable_variables()
regularization_cost = 0.001* tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv ])
cost = original_cost_function + regularization_cost

train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [12]:
with tf.name_scope("accuracy"):
    prediction = tf.argmax(logits,1)
    correct_prediction = tf.equal(prediction,y)
    correct_num=tf.reduce_sum(tf.cast(correct_prediction,tf.float32))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuracy")

In [13]:
X_train,X_test,Y_train,Y_test = train_test_split(word2vec_matrix,y_label,test_size = 0.2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(2000):
        _batch_size = 1000
        if len(listnum)<_batch_size:
            listnum = list(range(len(X_train)))
            Iter += 1
        temp_x =[]
        temp_y = []
        for _ in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        if (i)%200 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={_X:X_test, y: Y_test, keep_prob: 1.0, batch_size:len(X_test)})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy))
        sess.run(train_op,feed_dict={_X:temp_x,y:temp_y,keep_prob: 0.6, batch_size: _batch_size})

Iter 1: step 1, training accuracy 0.000943619
Iter 13: step 201, training accuracy 0.833215
Iter 26: step 401, training accuracy 0.848785
Iter 38: step 601, training accuracy 0.866006
Iter 51: step 801, training accuracy 0.86577
Iter 63: step 1001, training accuracy 0.87214
Iter 76: step 1201, training accuracy 0.866714


KeyboardInterrupt: 

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(word2vec_matrix,y_label,test_size = 0.2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(2000):
        _batch_size = 1000
        if len(listnum)<_batch_size:
            listnum = list(range(len(X_train)))
            Iter += 1
        temp_x =[]
        temp_y = []
        for _ in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        if (i)%200 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={_X:X_test, y: Y_test, keep_prob: 1.0, batch_size:len(X_test)})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy))
        sess.run(train_op,feed_dict={_X:temp_x,y:temp_y,keep_prob: 0.7, batch_size: _batch_size})

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(word2vec_matrix,y_label,test_size = 0.2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(2000):
        _batch_size = 1000
        if len(listnum)<_batch_size:
            listnum = list(range(len(X_train)))
            Iter += 1
        temp_x =[]
        temp_y = []
        for _ in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        if (i)%200 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={_X:X_test, y: Y_test, keep_prob: 1.0, batch_size:len(X_test)})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy))
        sess.run(train_op,feed_dict={_X:temp_x,y:temp_y,keep_prob: 0.8, batch_size: _batch_size})